In [1]:
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from elasticsearch import Elasticsearch
from faker import Faker

docker run -d \
  --name elastic \
  -p 9200:9200 \
  -e "discovery.type=single-node" \
  -v /home/apprenant/Documents/DEV_IA/NLP/NLP_Psy/Data:/usr/share/elasticsearch/data \
  docker.elastic.co/elasticsearch/elasticsearch:7.17.10


## Temps 2:  Stocker les données avec Elastic Search

### Mise en place
- Démarrer un container à partir de l’image docker.elastic.co/elasticsearch/elasticsearch:7.17.10
- en mode détaché -d
- monter le volume /usr/share/elasticsearch/data en local
- utiliser le mapping de port -p 9200:9200
- utiliser la variable -e "discovery.type=single-node"
- le nom du container sera --name elastic

- Visualiser les logs du container 
- appeler la route racine “/”


docker run -d --name elastic -p 9200:9200 -e "discovery.type=single-node" -v NLP_Psy/Data docker.elastic.co/elasticsearch/elasticsearch:7.17.10


docker run -d --name elastic 
-p 9200:9200 
-e "discovery.type=single-node" 
-v /home/apprenant/Documents/DEV_IA/NLP/NLP_Psy:/usr/share/elasticsearch/data docker.elastic.co/elasticsearch/elasticsearch:7.17.10


docker logs elastic

curl -X GET http://localhost:9200/


### Mapper et importer les données


Poster le mapping d’un index nommé “notes”, contenant :

- champ “patient_lastname” qui est un term
- champ “patient_firstname” qui est un term
- champ “text”, texte analysé en standard
- champ “date”, qui est une date
- champ “patient_left”, qui est un booléen
- champ “emotion”, qui est un term
- champ ‘confidence” qui est un float

Alimenter l’index “notes” à l’aide du jeu de données et de la librairie Faker. 

Bonus: Mettez en place un pipeline utilisant le modèle TF-IDF que vous avez développé avec scikit-learn pour remplir l- champs “emotion” et “confidence”.


In [13]:
es = Elasticsearch(hosts=["http://localhost:9200"])
es.transport.close()

In [5]:

# Connexion à Elasticsearch
es = Elasticsearch(hosts=["http://localhost:9200"])

# Mapping de l'index "notes"
mapping = {
    "mappings": {
        "properties": {
            "patient_lastname": {"type": "keyword"},
            "patient_firstname": {"type": "keyword"},
            "text": {"type": "text", "analyzer": "standard"},
            "date": {"type": "date"},
            "patient_left": {"type": "boolean"},
            "emotion": {"type": "keyword"},
            "confidence": {"type": "float"}
        }
    }
}

# Création de l'index avec le mapping
index_name = "notes"
es.indices.create(index=index_name, body=mapping)
es.transport.close()


/tmp/ipykernel_13195/3707423313.py:21: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index=index_name, body=mapping)
/tmp/ipykernel_13195/3707423313.py:21: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.create(index=index_name, body=mapping)


In [6]:
index_name = "notes"
df = pd.read_csv("Predicted_Data.csv")
fake = Faker()
es = Elasticsearch(hosts=["http://localhost:9200"])
# Alimentation de l'index "notes"
for index, row in df.iterrows():
    doc = {
        "patient_lastname": fake.last_name(),
        "patient_firstname": fake.first_name(),
        "text": row["Text"],
        "emotion": row["Emotion"],
        "date": fake.date(),
        "patient_left": fake.boolean(),
        "confidence": row['Confidence']
    }
    es.index(index=index_name, body=doc)
es.transport.close()

/tmp/ipykernel_13195/2487134383.py:16: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index=index_name, body=doc)
/tmp/ipykernel_13195/2487134383.py:16: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.index(index=index_name, body=doc)


ApiError: ApiError(429, 'cluster_block_exception', 'index [notes2] blocked by: [TOO_MANY_REQUESTS/12/disk usage exceeded flood-stage watermark, index has read-only-allow-delete block];')